In [1]:
import sys
sys.path.append("../ares-sc2")
sys.path.append("../src")

In [2]:
import pickle
import lzma
import numpy as np
import plotly.express as px

from sc2.position import Point2
from skimage.draw import disk
from phantom.cython.dijkstra_pathing import DijkstraPathing

ModuleNotFoundError: No module named 'phantom.cython.dijkstra_pathing'

In [ ]:
map_file = "../resources/maps/Acropolis AIE.xz"
with lzma.open(map_file, "rb") as f:
	map_data = pickle.load(f)

In [ ]:
np.random.seed(42)
pathing_grid = map_data.pathing_grid.data_numpy.T
pathing_cost = np.where(pathing_grid == 1.0, 1.0, np.inf)
cost = pathing_cost + np.exp(np.random.randn(*map_data.map_size))
# cost = pathing_cost
for _ in range(16):
	p = np.random.uniform((0, 0), map_data.map_size)
	r = np.random.uniform(4, 24)
	c = np.random.uniform(4, 24)
	d = disk(p, r)
	cost[d] += c
targets = [Point2((
	np.random.randint(0, map_data.map_size[0]),
	np.random.randint(0, map_data.map_size[1]),
)) for _ in range(8)]
rewards = [0, 0, 0, 0, 0, 0, 0, 100]
rewards

In [ ]:
paths = DijkstraPathing(cost, targets, rewards)

In [ ]:
img = paths.dist.copy()
for q in paths.get_path(map_data.map_center.rounded):
	img[q] = np.nan
px.imshow(img)

In [ ]:
%%timeit
paths = DijkstraPathing(cost, targets, rewards)
paths.prev_y